# Driver distraction classification using Convolutional Neural Networks

### Import libraries

In [7]:
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

### Data preprocessing

In [5]:
workingdir = os.path.abspath('')
print(workingdir)
csvdata = pd.read_csv(os.path.join(workingdir + '/state-farm-distracted-driver-detection//driver_imgs_list.csv'))
csvdata.head(2)

C:\Users\thami\Documents\GitHub\MachineLearningDriverdistraction


,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg


In [2]:
#Upload Training Data
c0directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c0')
c1directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c1')
c2directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c2')
c3directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c3')
c4directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c4')
c5directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c5')
c6directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c6')
c7directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c7')
c8directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c8')
c9directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c9')
print('total training c0 images:', len(os.listdir(c0directory)))
print('total training c1 images:', len(os.listdir(c1directory)))
print('total training c2 images:', len(os.listdir(c2directory)))
print('total training c3 images:', len(os.listdir(c3directory)))
print('total training c4 images:', len(os.listdir(c4directory)))
print('total training c5 images:', len(os.listdir(c5directory)))
print('total training c6 images:', len(os.listdir(c6directory)))
print('total training c7 images:', len(os.listdir(c7directory)))
print('total training c8 images:', len(os.listdir(c8directory)))
print('total training c9 images:', len(os.listdir(c9directory)))

C:\Users\thami\Documents\GitHub\MachineLearningDriverdistraction
total training c0 images: 2489
total training c1 images: 2267
total training c2 images: 2317
total training c3 images: 2346
total training c4 images: 2326
total training c5 images: 2312
total training c6 images: 2325
total training c7 images: 2002
total training c8 images: 1911
total training c9 images: 2129


In [ ]:
sample = cv2.imread(os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c0/img_34.jpg')

### Image generator

In [3]:
trainingdatadirectory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/')
trainingdataimage = ImageDataGenerator(rescale = 1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,
                                           shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

testdatadirectory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/test/')
testdataimage = ImageDataGenerator(rescale = 1./255)

trainingdatagenerator = trainingdataimage.flow_from_directory(trainingdatadirectory, target_size=(64,64),
                                                              class_mode='categorical', batch_size=128)

testdatagenerator = testdataimage.flow_from_directory(testdatadirectory, target_size=(64,64),
                                                      batch_size=128, class_mode=None, shuffle = False)


Found 22424 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


### Model

In [4]:
model = tf.keras.models.Sequential([
    
    # There are four convolutions
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])


model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          3

In [5]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
history = model.fit(trainingdatagenerator, epochs=25, steps_per_epoch=20, validation_data = testdatagenerator, 
                       verbose = 1, validation_steps=3)